In [1]:
#pulls verses from TaNaKh using the Sefaria API
#reads a query-formatted statement, and runs some functions from the text_converter

In [2]:
import json, urllib.request
from urllib.parse import urlparse, parse_qs

In [3]:
aleph_bet = ['א','ב','ג','ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם',
             'נ','ן','ס','ע','פ','ף','צ','ק','ר','ש','ת','װ','ױ','ײ','יִ',
             'ﬡ','ﬢ','ﬣ','ﬤ','ﬥ','ﬦ','ﬧ','ﬨ','שׁ','שׂ','שּׁ','שּׂ','אַ','אָ',
             'גּ','דּ','הּ','וּ','זּ','טּ','יּ','ךּ','כּ','לּ','מּ','נּ','סּ','ףּ',
             'פּ','צּ','שּ','תּ','וֹ','בֿ','כֿ','פֿ','ﭏ','בּ', 'קּ']

cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

cant2 = cant[:28]

vowel = ['ְ','ֱ','ֲ','ֳ','ִ','ֵ','ֶ','ַ','ָ','ֹ','ֺ','ֻ','ּ','ֽ','־','ֿ','ׁ','ׂ','ׄ','ׅ','ׇ']

letter_with_dagesh = ['שּׁ','שּׂ','גּ','דּ','זּ','טּ','יּ','כּ','לּ','מּ','נּ','סּ','פּ','צּ','שּ','תּ','בּ','קּ']
rafe = 'ֿ'
shva = vowel[0]
chataf_vowels = vowel[1:4]
short_vowels = [vowel[4],vowel[6],vowel[7],vowel[11],vowel[20]]
long_vowels = ['וֹ','וּ', vowel[5], vowel[8], aleph_bet[9], 'ֹ']
vowels_limited = vowel[0:4]+short_vowels+long_vowels
dagesh = 'ּ'
maqqaf = '־'
meteg = 'ֽ'
dot = '֯'
silluq = '׃'
colon = ':'
other_char = ['ׂ','ׁ','ֿ','֯','ׅ','']
all_char = aleph_bet+cant+vowel+other_char

special_characters = ['{', '}','\\']

gutterals = ['א','ה','ח','ע','ר' ,'ﬡ','ﬣ','ﬧ']    

In [4]:
letters_w_final = ['כ','מ','נ','פ','צ']
final_letters = ['ך','ם','ן','ף','ץ']

In [5]:
shem = 'יהוה'
shem_vowels = 'יְהֹוָה'
yy_vowels = aleph_bet[9]+vowel[0]+aleph_bet[9]+vowel[8]

kal_backwards_dagesh = 'כָּל'
kal_dagesh = 'כָּל'
kal = aleph_bet[52] + vowel[8] + aleph_bet[12]
khal = aleph_bet[10] + vowel[8] + aleph_bet[12]
kol = aleph_bet[52] + vowel[20] + aleph_bet[12]
kol_maqqaf = aleph_bet[52] + vowel[20] + aleph_bet[12] + '־'
khol_maqqaf = aleph_bet[10] + vowel[20] + aleph_bet[12] + '־'
kol_space = aleph_bet[52] + vowel[20] + aleph_bet[12] + ' '
khol_space = aleph_bet[10] + vowel[20] + aleph_bet[12] + ' '

et = 'אֶת'
ve_et = 'וְאֶת'
space_et_space = ' ' + 'אֶת' + ' '
space_ve_et_space = ' ' + 'וְאֶת' + ' '
et_maqqaf = ' ' + 'אֶת' + '־'
ve_et_maqqaf = ' ' + 'וְאֶת' + '־'

In [6]:
shva_exceptions = ['שְׁתֵּי','שְׁתָּיִם','שְׁתַּיִם','שְׁנִַים','שְׁנֵי','שְׁתֵּים','שְׁנֵים','שְׁנֵי','שְׁנֵים','שְׁנָיִם']
battim = 'בָּתִּים'
batteinu = 'בָתֵּֽינוּ'
vattim = 'בָתִּים'
battei = 'בָּתֵּי'
ana = 'אָנָּא'
ana2 = 'אָנָּא'
ana3 = 'אָנָּא'
anah = 'אָנָּה'
kamatz_katan_exceptions = [battim, vattim, ana, ana2, ana3, anah, batteinu]
known_nasog_achor = ['היתה','אזרו']
known_shva_nach = ['לילה']

In [7]:
#removes trop and vowels
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [8]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [9]:
#converts the 4-letter shem to double-yod, preserving trop and prefixes

def shem_converter(word):
    prefix = ''
    if word[0] != aleph_bet[9]:
        prefix = prefix + word[0]
        i = 1
        if word[1].isalpha() == False:
            prefix = prefix + word[1]
            if word[2].isalpha() == False:
                prefix = prefix + word[2]
    no_prefix_word = word[len(prefix):]

    if len(prefix) > 0:
        yud1 = aleph_bet[9]
    else:
        yud1 = aleph_bet[9]+vowel[0]

    yud2 = aleph_bet[9]+vowel[8]

    cant_marks = []

    #finds suffix
    suffix = ''
    i = -1
    while word[i] != aleph_bet[4]:
        if word[i] in special_characters:
            suffix = word[i] + suffix
        i = i-1
    
    for character in no_prefix_word:
        if character in cant:
            cant_marks.append(character)  
    if len(cant_marks) == 1:
        new_shem = prefix + yud1 + yud2 + cant_marks[0] + suffix
    elif len(cant_marks) == 2:
        new_shem = prefix + yud1 + cant_marks[0] + yud2 + cant_marks[1] + suffix
    else:
        new_shem = prefix + yud1 + yud2 + suffix

    return new_shem

In [10]:
#creates paragraph with new shem using function above
#this needs to be fixed to stop eating special characters (i.e. brackets, \)

def convert_shem(paragraph):
    paragraph = str.replace(paragraph, '־', ' ־ ')
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        if shem in nonalpha_remover(word):
            double_yud = shem_converter(word)
            par_list[index] = double_yud

    new_par = ' '.join(par_list)
    new_par = str.replace(new_par, ' ־ ','־')
    new_par = str.replace(new_par, '׃', '׃')
    return new_par

In [11]:
def na_marker(paragraph,index, mark):
    if paragraph[index-1] == mark or paragraph[index+1] == mark or paragraph[index+2] == mark or paragraph[index-2]==mark:
        return paragraph
        #does nothing, if the shva is already marked with a rafe
    else:
        par_start = paragraph[:index]
        par_end = paragraph[index:]
        new_par = par_start+mark+par_end
        return new_par
        #adds a rafe over the input letter

In [12]:
def long_vowel_checker(word,i,index):
    previous_vowel = word[i]
    while word[i] not in aleph_bet:
        i = i-1
    word_sec = word[i:index]
    #if there's no trop and the previous syllable has a meteg, it's a long vowel with primary stress and the shva is nach
    if any(trop in cant2 for trop in word)==False and meteg in word_sec:
        return False
    trop_count = 0
    for char in word:
        if char in cant2:
            trop_count = trop_count+1
    #if there's 2 trops and 1 is on the previous syllable, the 2nd will be on the last syllable
    #therefore the 1st will be a long vowel with secondary stress, and the shva will be na
    #and this second trope will be on the last syllable, so the shva following a syllable with a trop will be the first one
    #and stress will be secondary, so the shva will be na
    if trop_count == 2 and any(trop in cant2 for trop in word_sec)==True:
        return True
    if trop_count == 2 and any(trop in cant2 for trop in word_sec)==False:
        return True
    #if there's one trop on the word and it's not on the previous syllable, it's unstressed, and the shva is na
    elif trop_count == 1 and any(trop in cant2 for trop in word_sec)==False:
        return True
    #if there's a trop on the previous syllable, assume it's stressed and nach, but it could be nasog achor and na
    #therefore these will be printed for manual checking
    elif trop_count == 1 and any(trop in cant2 for trop in word_sec)==True:
        bare_word = nonalpha_remover(word)
        for element in known_nasog_achor:
            if element in bare_word:
                return True
        for element in known_shva_nach:
            if element in bare_word:
                return True
        print('Check shva in',word,'for nasog achor')
        return False
    elif trop_count == 0 and meteg not in word_sec:
        return True
    else:
        print('Check shva in',word)

In [13]:
#Determines what shvas are na' in a word, and calls the program to mark them
def shva_na_function(word, mark):
    nt_word = trop_remover(word)
    #this skips instances of shem hashem
    if shem_vowels in nt_word or yy_vowels in nt_word:
        return word
    for excep in shva_exceptions:
        if excep in nt_word:
            return word
    if shva not in word:
        return word
    start = 1
    ha_meteg = 'הַֽ'
    shuruk_meteg = 'וּֽ'
    if word[0:3] == ha_meteg or word[0:3] == shuruk_meteg:
        shva_loc = word.find(shva)
        word = na_marker(word,shva_loc, mark)
        start = shva_loc+1
    
    for index in range(start,len(word)-2):
        i = index
        if word[index] != shva:
            continue
            #this doesn't bother with the loop if the letter isn't a shva
        if word[i+1] not in all_char:
            break
        while word[i] not in aleph_bet or word[i]== dagesh:
            i = i-1
        previous_consonant = word[i]
        #if the previous consonant is the beginning of the word, the shva is na'
        if i == 0:
            word = na_marker(word,index, mark)
            continue
        if word[i-1] not in all_char:
            word = na_marker(word,index, mark)
            continue
        
        if word[i-1] == maqqaf:
            word = na_marker(word,index, mark)
        #if the previous consonant has a dagesh, the shva must be na'
        if previous_consonant in letter_with_dagesh:
            word = na_marker(word,index, mark)
            continue
        if word[i+1] == dagesh or word[index+1] == dagesh:
            word = na_marker(word,index, mark)
            continue
            
        #determines the next consonant
        i = index + 1
        while word[i] not in aleph_bet and i<len(word)-1:
            i = i+1
        next_consonant = word[i]
        if next_consonant == previous_consonant:
            word = na_marker(word,index, mark)
            continue
        i=0
        while i<5:
            if word[index]==letters_w_final[i] and next_consonant == final_letter[i]:
                word = na_marker(word,index, mark)
                continue   
            i=i+1

        #determines the previous vowel
        i = index-2
        while word[i] not in vowels_limited and i>0:
            i = i-1
        if i == 0:
            continue
        else:
            previous_vowel = word[i]
            #if the previous vowel is a shva, the current shva is na'
            if previous_vowel == shva:
                word = na_marker(word,index, mark)
                continue
            if previous_vowel in long_vowels and long_vowel_checker(word,i,index)==True:
                word = na_marker(word,index, mark)   
        
        #a shva following a long vowel is na', unless the long vowel is word-initial shuruk
   

    #figure out plan for סקינמלוי letters after הַ־ without dagesh
    #figure out plan for letters following shuruk with meteg
    #for now, neither of these get a shva na
    return word

In [14]:
#Calls the shva_na_function for each word in the input paragraph
def shva_na_converter(paragraph, mark):
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        par_list[index] = shva_na_function(word, mark)
    new_par = ' '.join(par_list)
    return new_par

In [15]:
## Puts a kamatz katan in common kamatz-katan words and situations

In [16]:
#removes shva marks, to check for kamatz katans
#Since some may preceed erroneously marked shvas
#This will need to be adjusted if the shva na marker is not a rafe
def shva_mark_remover(word):
    if rafe in word:
        word = word.replace(rafe,'')
    if dot in word:
        word = word.replace(dot,'')
    return word

In [17]:
#this imports a list of kamatz-katan words to fix and their replacements
#import csv
import csv
words_to_fix = []
corrected_words = []
with open('kamatz_correction_list.csv', 'r', encoding='utf-8') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        words_to_fix.append(shva_mark_remover(row[0]))
        corrected_words.append(row[1])
words_to_fix=words_to_fix[1:]
corrected_words=corrected_words[1:]

In [18]:
def kamatz_list_tester(word):
    i=0
    while i<len(words_to_fix):
        if shva_mark_remover(word)==words_to_fix[i]:
            word = corrected_words[i]
        i=i+1
    return word

In [19]:
def kamatz_katan_dagesh_tester(word,index,i):
    next_consonant = word[i]
    if next_consonant == 'ה':
        return False
    elif word[index+1] == meteg:
        return False
    elif next_consonant in letter_with_dagesh:
        return True
    elif word[i+1] == dagesh:
        return True
    else:
        return False

In [20]:
def kamatz_katan_adder(word):    
    #prevents script from erroneously fixing false positives
    nt_word = nonalpha_remover(word)
    for index in range(0,len(kamatz_katan_exceptions)):
        if nonalpha_remover(kamatz_katan_exceptions[index]) in nt_word:
            return word
    
    #runs word through list of predefined kamatz words to check
    if shva_mark_remover(word) in words_to_fix:
        return kamatz_list_tester(word)
    elif (shva_mark_remover(word)+colon or shva_mark_remover(word)+silluq) in words_to_fix:
        return kamatz_list_tester(word)+silluq
    
    for index in range(1,len(word)-3):
        if word[index] == vowel[8]:
            i = index
            while word[i] not in aleph_bet:
                i = i+1
                if i>len(word)-1:
                    break
            if i<len(word)-1:
                next_consonant = word[i]
                if kamatz_katan_dagesh_tester(word,index,i)==True:
                    word = word[:index]+vowel[20]+word[index+1:]
            
            i = index+1
            while word[i] not in vowels_limited:
                i = i+1
                if i>len(word)-1:
                    return word
            next_vowel = word[i]
            if next_vowel == chataf_vowels[2] and next_consonant in gutterals:
                word = word[:index]+vowel[20]+word[index+1:]
                return word
            #finds the vowel after the next one.  If both are shvas, the kamatz is katan.
            if next_vowel == shva and shva in word[index+1:]:
                if i<len(word)-1:
                    i = i+1
                    while i<len(word)-2 and word[i] not in vowels_limited:
                        i = i+1
                    sec_vowel = word[i]
                    if sec_vowel == shva:
                        word = word[:index]+vowel[20]+word[index:]
    return word

In [21]:
def kamatz_katan(paragraph):
    #This goes through the list of common words with kamatz katan (besides "kol")
    #and corrects them if they are present.
    #This list can be added to as needed.
    par_list = paragraph.split()
    
    #Calls the kamatz_katan_adder for each word in paragraph, if that word has a kamatz at all
    for index in range(len(par_list)):
        if vowel[8] in par_list[index]:
            par_list[index] = kamatz_katan_adder(par_list[index])
    paragraph = ' '.join(par_list)
    return paragraph  

In [22]:
#this script does not work on words with trope
#Since pesukim from wikisource have maqqafs, this shouldn't be a problem.

def et_fixer(paragraph):
    par_list = paragraph.split()
    if et in par_list:
        new_par_split = paragraph.split(space_et_space)
        new_paragraph = et_maqqaf.join(new_par_split)
        new_par_split = new_paragraph.split(space_ve_et_space)
        new_paragraph = ve_et_maqqaf.join(new_par_split)
    else:
        new_paragraph = paragraph
    return new_paragraph

In [23]:
#changes colon to a silluq character
def colon_changer(line):
    #this avoids changing colons in instructions to silluk, as these should remain colons
    if 'instruction' in line:
        return line
    else:
        return line.replace(colon,silluq)

In [24]:
#this changes chataf-patach to shva when there's different conventions to when chataf-patach.
#this script finds chataf-patach not under gutteral letters, and changes it to a shva
def chataf_shva_swapper(word):
    chataf_patach = 'ֲ'
    if chataf_patach in word:
        for index in range(1,len(word)-3):
            i = index
            if word[index] != chataf_patach:
                continue

            while word[i] not in aleph_bet:
                i = i-1
            previous_consonant = word[i]
            if previous_consonant in gutterals[0:4]+gutterals[6:7]:
                continue
            else:
                word = word[:index]+shva+word[index+1:]
    return word

#Calls the shva_na_function for each word in the input paragraph
def chataf_converter(paragraph):
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        par_list[index] = chataf_shva_swapper(word)
    new_par = ' '.join(par_list)
    return new_par

In [25]:
def text_converter(paragraph):
    #Comment out components of the script you don't want to run
    #paragraph = shva_mark_remover(paragraph)
    #functions that only are needed for what's not copied from MAPM:
    paragraph = shva_na_converter(paragraph,rafe)
    if any(trop in cant2 for trop in paragraph)==False:
        paragraph = kol_kamatz_katan(paragraph)
        paragraph = et_fixer(paragraph)
        paragraph = kamatz_katan(paragraph)
        paragraph = chataf_converter(paragraph)
    return paragraph

In [26]:
###the following is the code to pull verses
#it uses some of the converters above, but not all are implemented.

In [27]:
shortnames = {
    "מלכים א":"מ״א",
    "מלכים ב":"מ״ב",
    "שמואל א":"ש״א",
    "שמואל א":"ש״ב",
    "בראשית":"ברא׳",
    "ישעיהו":"ישעיה",
    "ירמיהו":"ירמיה",
    "יחזקאל":"יחזקﭏ",
    "שיר השירים":"שה״ש",
    "דברי הימים א":"דה״א",
    "דברי הימים ב":"דה״ב"
}

def shortentitle(title):
    for entry in shortnames:
        if entry in title:
            title = title.replace(entry,shortnames[entry])
    if title[-1] == "׳":
        title = title[0:-1]
    if title[-2] == "״":
        title = title[0:-2]+title[-1]
    return title

In [28]:
def textpuller(text,version):
    link = "https://www.sefaria.org/api/texts/"+text+"?vhe="+version+"&context=0"
    print(link)
    with urllib.request.urlopen(link) as url:
        sefariaresult = json.loads(url.read().decode())
    text = ""
    if type(sefariaresult["he"])==str:
        text = sefariaresult["he"]
    elif type(sefariaresult["he"])==list:
        if type(sefariaresult["he"][0])==str:
            for verse in sefariaresult["he"]:
                text = text + verse + " "
        elif type(sefariaresult["he"][0])==list:
            textblock = []
            for chunk in sefariaresult["he"]:
                textblock = textblock + chunk
            for verse in textblock:
                text = text + verse + " "
    title_he = sefariaresult["heSectionRef"]
    return text,title_he

#figure out how to deal with formatting

In [29]:
def removeformatting(text):
    while "<" in text and ">" in text:
        loc1 = text.find("<")
        loc2 = text.find(">",loc1)+1
        text = text.replace(text[loc1:loc2],"")
    return text

In [30]:
def qkparse(string):
    start = string.index('<span class=\"mam-kq">')
    string_before = string[0:start]
    startk = string.index('<span class=\"mam-kq-k\">',start)
    startk = string.index('>',startk)+2
    endk = string.index('</span>',startk)-1
    ktext = string[startk:endk]
    startq = string.index('<span class=\"mam-kq-q\">')
    startq = string.index('>',startq)+2
    endq = string.index('</span>',startq)-1
    qtext=string[startq:endq]
    end = string.index('</span></span>',endq)
    end = string.index(' ',end)
    output = '\\qk{'+qtext+'}{'+ktext+'}'
    string_after = string[end:]
    return string_before+output+string_after

In [31]:
##using the above to generate a commands

In [32]:
def parserequest(string):
    if len(string)<3:
        return string
    if string[-1] == "\n":
        string = string[0:-1]
    while string[-1] == " ":
        string = string[0:-1]
    parsed_query = urlparse(string)
    parameters = parse_qs(parsed_query.query)
    if parameters == {}:
        return string
    text = parameters["text"][0]
    if "version" not in parameters.keys():
        version = ""
    elif parameters["version"][0] == 'MAPM':
        version = "Miqra_according_to_the_Masorah"
    else:
        version = parameters["version"][0]
    
    sefariaresult = textpuller(text,version)
    verses = sefariaresult[0]
    title = shortentitle(sefariaresult[1])
    
    #these run text converters, if the relevant items are listed in the query under "convert"
    if "convert" in parameters.keys():
        if 'qk' in parameters["convert"]:
            while '<span class=\"mam-kq\">' in verses:
                verses = qkparse(verses)
        if "removeHTML" in parameters["convert"]:
            verses = removeformatting(verses)
        if "shemyy" in parameters["convert"]:
            verses = convert_shem(verses)
        if "shva" in parameters["convert"]:
            verses = shva_na_converter(verses, rafe)
       
    #if there's a silluq, this truncates what's after it
    #mostly useful for getting rid of parsha-break notes at the end of the quoted text
    if "׃" in verses:
        end = verses.rfind("׃")+1
        verses = verses[0:end+1]
    
    outputstring = ""
    if "name" in parameters.keys():
        outputstring += "\\newcommand{\\"+parameters["name"][0]+"}{%\n"
        
    if "source" in parameters:
        if parameters["source"][0] == "True":
            sourcestring = "\source{"+title+"}"
            first_space = verses.find(" ")
            outputstring += verses[0:first_space]+sourcestring+verses[first_space:-1]
        elif parameters["source"][0] == "footnotestart":
            sourcestring = "\footnote{"+title+"}"
            outputstring = sourcestring + outputstring
        elif parameters["source"][0] == "footnoteend":
            sourcestring = "\footnote{"+title+"}"
            outputstring = outputstring + sourcestring
        elif parameters["source"][0] == "instruction":
            sourcestring = "\instruction{"+title+"}\n"
            outputstring = sourcestring + outputstring

    else:
        outputstring += verses
       
    if "name" in parameters.keys():
        outputstring += "}"
    
    return outputstring

In [33]:
#runs the parser for each line of a TeX file
def TeXconverter(inputfile,outputfile):
    with open(inputfile, 'r', encoding='utf-8') as infile:
        lines = list(infile.readlines())
    output_lines = []
    for line in lines:
        if line[-1]=="\n":
            output_lines.append(parserequest(line)+"\n")
        else:
            output_lines.append(parserequest(line))
    with open(outputfile, 'w', encoding='utf-8') as outfile:
        for line in output_lines:
            outfile.write(line)

In [34]:
#query options:
#text: book/chapter/verse to print.
#works for tanakh and other texts
#sefaria is a bit picky about formatting, capitalize first letters.
#books in 2 sections in tanakh need to be 1_Kings etc

#version--optional.  the version of the text to pull.
#MAPM is expanded automatically to "Miqra According to the Masora"

#convert determines what text converters to run the text through.
#you can have multiple &convert=... statements
#qk puts a kri ukhtiv into a LaTeX command \qk{}{}
#removeHTML removes the formatting statements in <>
#shemyy converts the 4-letter shem to double-yod
#shva adds marking for shva na using a rafe

#name-turns the pull statement into a LaTeX new command, which is named \name

#source--set &source=True to add a LaTeX statement that adds \source{} with the book/chapter in Hebrew after the first word
#footnotestart and footnoteend add a foornote before or after the text with the source
#instruction adds the text title in an \instruction{} before the body of the text

In [55]:
#sample implementations:
test = "%?text=Berakhot.3a&convert=shva"
test = "%?text=Nehemiah.9:6&convert=shemyy&convert=qk&version=MAPM&&source=True&convert=removeHTML \n"
parserequest(test)
#parserequest(test2)

https://www.sefaria.org/api/texts/Nehemiah.9:6?vhe=Miqra_according_to_the_Masorah&context=0


'אַתָּה־ה֣וּא\\source{נחמיה ט} יְיָ לְבַדֶּ֒ךָ֒ \\qk{אַתָּ֣ה}{את} עָשִׂ֡יתָ אֶֽת־הַשָּׁמַ֩יִם֩ שְׁמֵ֨י הַשָּׁמַ֜יִם וְכׇל־צְבָאָ֗ם הָאָ֜רֶץ וְכׇל־אֲשֶׁ֤ר עָלֶ֙יהָ֙ הַיַּמִּים֙ וְכׇל־אֲשֶׁ֣ר בָּהֶ֔ם וְאַתָּ֖ה מְחַיֶּ֣ה אֶת־כֻּלָּ֑ם וּצְבָ֥א הַשָּׁמַ֖יִם לְךָ֥ מִשְׁתַּחֲוִֽים'

In [74]:
#TeXconverter('weekday_torah_readings_raw.tex','weekday_torah_readings.tex')

https://www.sefaria.org/api/texts/Genesis.1:1-5?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.1:6-8?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.1:9-13?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.6:9-16?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.6:17-19?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.6:20-22?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.12:1-3?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.12:4-9?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.12:10-13?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.18:1-5?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Genesis.18:6-8?vhe=Miqra

https://www.sefaria.org/api/texts/Leviticus.19:1-4?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.19:5-10?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.19:11-14?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.21:1-6?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.21:7-12?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.21:13-15?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.25:1-3?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.25:4-7?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.25:8-13?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Leviticus.26:3-5?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts

https://www.sefaria.org/api/texts/Numbers.28:4-5?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Numbers.28:6-10?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Numbers.28:11-15?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.13:1-4?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.13:5-10?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.13:11-16?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.22:24-26?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.22:27-23:5?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.23:6-19?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.34:1-10?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Exodus.34:11-17?vhe